In [1]:
import csv
import pandas as pd
import numpy as np
import math
import pickle
import networkx as nx
import pickle
from collections import defaultdict

In [2]:
NUM_ROWS = 20000
following = defaultdict(list)
followed = defaultdict(list)

In [3]:
with open('data/train.txt', 'r') as f:
    for i in range(NUM_ROWS): 
        if f:
            line = next(f)
            nodes = line.split('\t')
            source = int(nodes[0])
            sinks = list(map(int, nodes[1:]))
            
            # disregard those followed by too many people
            if len(sinks) < 2500:
                following[source] = sinks 

                # aggregate followers of users
                for n in sinks:
                    followed[n].append(source)
                
    f.close()

In [4]:
count_popular = 0
to_remove = []
for key, value in followed.items():
    if len(value) > 1:
        count_popular += 1
    else:
        to_remove.append(key)

In [5]:
count_popular

587704

In [6]:
len(to_remove)

776833

In [7]:
from itertools import product

full_graph = nx.DiGraph()
nodes = list(following.keys())

for n in nodes:
    targets = following[n]
    combinations = list(product([n], targets))
    full_graph.add_edges_from(combinations)

In [8]:
print(nx.info(full_graph))

Name: 
Type: DiGraph
Number of nodes: 1364537
Number of edges: 5999507
Average in degree:   4.3967
Average out degree:   4.3967


In [9]:
nx.write_edgelist(full_graph, 'filtered_edgelist', data=False)

In [10]:
from random import sample

NUM_SAMPLE = 50000

# sample 50000 edges
true_edges = sample(full_graph.edges, NUM_SAMPLE)

In [60]:
false_edges = []
count_edge = 0
all_nodes = list(full_graph.nodes)

In [61]:
from random import choice
while count_edge < NUM_SAMPLE:
    # get a random node to start
    source = choice(nodes)
    # find missing edges from 2 distance
    one_neighbors = list(full_graph.neighbors(source))
    count_n1 = 0
    for n1 in one_neighbors:
        if count_n1 > 100:
            count_edge += count_n1
            break
        else:
            count_n2 = 0
            two_neighbors = list(full_graph.neighbors(n1))
        # get up to 100 missing edges from source
        for n2 in two_neighbors:
            if count_n2 > 10:
                count_n1 += count_n2
                break
            if not full_graph.has_edge(source, n2):
                count_n2 += 1
                false_edges.append((source, n2))

In [63]:
def assign_label(pair, graph):
    u, v = pair[0], pair[1]
    return (int(graph.has_edge(u, v)))

def concatenate(node_set, label):
    dataset = pd.DataFrame({'nodes': node_set, 'label': label})
    return dataset

from functools import partial

In [64]:
# Label creation
y_1 = list(map(partial(assign_label, graph=full_graph), true_edges))
y_0 = list(map(partial(assign_label, graph=full_graph), false_edges))

In [67]:
# append y_true to y_train & true_edges to train_small
y_train = y_1 + y_0
edges_train = true_edges + false_edges

In [68]:
train_df = concatenate(edges_train, y_train)

In [ ]:
train_df.to_csv("small_train.csv", index=False)